In [1]:
import os
import shutil

from azureml.core.workspace import Workspace
from azureml.core import Experiment
from azureml.core import Environment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

In [2]:
ws = Workspace.from_config()
ws

Workspace.create(name='resnet', subscription_id='92c76a2f-0e1c-4216-b65e-abf7a3f34c1e', resource_group='azureml_uw_resnet')

In [3]:
cluster_name = "ML-CPU-test"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('No compute cluster named {}'.format(cluster_name))
    exit()

Found existing compute target


In [4]:
curated_env_name = 'Resnet50v15-CPU-cluster'
#pytorch_env = Environment.get(workspace=ws, name=curated_env_name)
pytorch_env = Environment.from_conda_specification(name=curated_env_name, file_path='./conda_dependencies.yml')

In [7]:
from azureml.core import ScriptRunConfig
from azureml.core.datastore import Datastore
from azureml.core import Dataset

project_folder = './'
data_path = './'

datastore = Datastore.get(ws, 'tiny_imagenet')
dataset = Dataset.File.from_files(path=(datastore, data_path))

src = ScriptRunConfig(source_directory=project_folder,
                      script='train_resnet.py',
                      arguments=[
                          '--num_epochs', 16,
                          '--shuffle', 'False',
                          '--dataloc', data_path,
                          '--output_dir', './outputs',
                          ],
                      compute_target=compute_target,
                      environment=pytorch_env)

DatasetValidationError: DatasetValidationError:
	Message: Cannot load any data from the specified path. Make sure the path is accessible and contains data.
ScriptExecutionException was caused by StreamAccessException.
  StreamAccessException was caused by AuthenticationException.
    Identity authentication failed for 'AzureBlob List' operation at '[REDACTED]' with '403: AuthenticationFailed'. Please make sure the compute or login identity has 'Storage Blob Data Reader' or 'Storage Blob Data Owner' role in the storage IAM.
      Failed due to inner exception of type: StorageException
| session_id=44932087-9c9a-432f-a828-543b7d465c74
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Cannot load any data from the specified path. Make sure the path is accessible and contains data.\nScriptExecutionException was caused by StreamAccessException.\n  StreamAccessException was caused by AuthenticationException.\n    Identity authentication fa

In [ ]:
run = Experiment(ws, name='Train-Resnet50v15').submit(src)
run.wait_for_completion(show_output=True)